In [1]:
import dash
from dash import dcc, html
import pandas as pd
import requests
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import time

In [2]:
app = dash.Dash(__name__)

In [3]:
app.layout = html.Div([
    html.H1("Real-Time Stock Price Dashboard"),
    dcc.Dropdown(
        id='stock-dropdown',
        options=[
            {'label': 'Apple', 'value': 'AAPL'},
            {'label': 'Google', 'value': 'GOOGL'},
            {'label': 'Amazon', 'value': 'AMZN'}
        ],
        value='AAPL'
    ),
    dcc.Graph(id='live-graph'),
    dcc.Interval(
        id='interval-component',
        interval=10*1000,  # in milliseconds
        n_intervals=0
    )
])

In [4]:
def fetch_stock_data(symbol):
    api_key = 'YOUR_API_KEY'  # Replace with your Alpha Vantage API key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    return data

In [6]:
@app.callback(
    Output('live-graph', 'figure'),
    [Input('stock-dropdown', 'value'),
     Input('interval-component', 'n_intervals')]
)
def update_graph(selected_stock, n):
    data = fetch_stock_data(selected_stock)
    time_series = data['Time Series (1min)']

    # Prepare data for plotting
    times = list(time_series.keys())
    prices = [float(time_series[time]['1. open']) for time in times]

    # Create the figure
    figure = go.Figure()
    figure.add_trace(go.Scatter(x=times, y=prices, mode='lines+markers', name=selected_stock))
    figure.update_layout(title=f'Real-Time Stock Price for {selected_stock}', xaxis_title='Time', yaxis_title='Price (USD)')

    return figure

In [7]:
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# Define a function to fetch stock data
def fetch_data(ticker):
    stock_data = yf.download(ticker, period="1d", interval="2m")
    return stock_data
